In [2]:
setwd('C:/Users/Eddie/Documents/GitHub/ML-Ex-Rates/Results')
library(rpart)
library(randomForest)
library(dplyr)

test_full_trees <- function(country, B){
    full <- read.csv(paste(country,'_full.csv',sep=''))
    full <- full[,-1]
    rownames(full) <- 1:nrow(full)
    
    # Split into testing and training
    n <- dim(full)[1]
    n1 <- round(n/5)
    
    # full Tree Models
    set.seed(19890211)
    fulltree.TE_list <- vector(length=0)
    for(i in 1:B){
        start <- sample(1:(n-n1-1),1)
        flag <- (start+6):(start+n1-6)
        train <- full[-flag,]
        test <- full[flag,]
        
        treemodel <- rpart(Pct_Chg ~ ., method="anova",data = train)
        fulltree.test <- predict(treemodel,test[,-1])
        fulltree.TE <- mean((test[,1]-fulltree.test)^2)
        fulltree.TE_list <- c(fulltree.TE_list, fulltree.TE)
    }
    
    # pruned tree models
    set.seed(19890211)
    fullpruned.TE_list <- vector(length=0)
    for(i in 1:B) {
        start <- sample(1:(n-n1-1),1)
        flag <- (start+6):(start+n1-6)
        train <- full[-flag,]
        test <- full[flag,]
        
        treemodel <- rpart(Pct_Chg ~ ., method="anova",data = train)
        prunedtree <- prune(treemodel,
                    treemodel$cptable[which.min(treemodel$cptable[,'xerror']),"CP"])
        fullpruned.test <- predict(prunedtree,test[,-1])
        fullpruned.TE <- mean((test[,1]-fullpruned.test)^2)
        fullpruned.TE_list <- c(fullpruned.TE_list,fullpruned.TE)  
    }
    
    # Full Random Forest
    set.seed(19890211)
    fullforest.TE_list <- vector(length=0)
    for(i in 1:B) {
        start <- sample(1:(n-n1-1),1)
        flag <- (start+6):(start+n1-6)
        train <- full[-flag,]
        test <- full[flag,]
        
        forestmodel <- randomForest(Pct_Chg ~ ., data = train)
        fullforest.test <- predict(forestmodel,test[,-1])
        fullforest.TE <- mean((test[,1]-fullforest.test)^2)
        fullforest.TE_list <- c(fullforest.TE_list, fullforest.TE) 
        
    }

output1 <- c('Full','Full Tree','Full Pruned', 'Full Forest')
output1 <- rbind(output1,c('Mean',mean(fulltree.TE_list),mean(fullpruned.TE_list),mean(fullforest.TE_list)))
output1 <- rbind(output1,c('Variance',var(fulltree.TE_list),var(fullpruned.TE_list),var(fullforest.TE_list)))


return(output1)

}

print('canada')
test_full_trees('canada',10)
print('europe')
test_full_trees('europe',10)
print('mexico')
test_full_trees('mexico',10)
print('japan')
test_full_trees('japan',10)
print('korea')
test_full_trees('korea',10)

randomForest 4.6-10
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'dplyr'

The following object is masked from 'package:randomForest':

    combine

The following object is masked from 'package:stats':

    filter

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00286186754928327,0.00272616818424158,0.00371842861805248
,Variance,9.73573559258191e-06,1.00597794608522e-05,2.89904602671658e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0141188413996312,0.0145516771244521,0.00887244130575673
,Variance,2.14367588826176e-05,3.0844576614562e-05,2.27384581385476e-05


...

output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0077701432084026,0.00762001524070815,0.00513111309502257
,Variance,6.87165671409264e-05,7.24558380221787e-05,2.51513461688025e-05


[1] "japan"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.030954847425092,0.0320798515009658,0.0172915545739086
,Variance,0.00015674290687077,0.000166866566216661,5.54138475996108e-05


[1] "korea"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0107077822435343,0.0104522252175932,0.0115367621337741
,Variance,8.12971069771746e-05,8.1545096726855e-05,0.000113930868570151


In [3]:
test_aic_trees <- function(country, B){    
    full <- read.csv(paste(country,'_full.csv',sep=''))
    full <- full[,-1]
    rownames(full) <- 1:nrow(full)

    #AIC Model
    regmodel <- lm(Pct_Chg ~ ., data = full)
    min.model = lm(Pct_Chg ~ 1, data = full)
    max_model <- formula(regmodel)
    linreg_AIC = step(min.model, direction = 'forward',
             scope = max_model, trace = FALSE)
    vars <- as.character(formula(linreg_AIC))
    var_list <- unlist(strsplit(vars[3],' +'))
    var_list <- var_list[c(TRUE,FALSE)]
    var_list <- c('Pct_Chg',var_list)
    aic_vars <- full[var_list]

    # Split into testing and training
    n <- dim(full)[1]
    n1 <- round(n/5)

    # AIC Tree Models
    set.seed(19890211)
    AICtree.TE_list <- vector(length=0)
    for(i in 1:B){
        start <- sample(1:(n-n1-1),1)
        flag <- (start+6):(start+n1-6)
        train <- aic_vars[-flag,]
        test <- aic_vars[flag,]

        treemodel <- rpart(Pct_Chg ~ ., method="anova",data = train)
        AICtree.test <- predict(treemodel,test[,-1])
        AICtree.TE <- mean((test[,1]-AICtree.test)^2)
        AICtree.TE_list <- c(AICtree.TE_list, AICtree.TE)
    }

    # AIC pruned tree models
    set.seed(19890211)
    AICpruned.TE_list <- vector(length=0)
    for(i in 1:B) {
        start <- sample(1:(n-n1-1),1)
        flag <- (start+6):(start+n1-6)
        train <- aic_vars[-flag,]
        test <- aic_vars[flag,]

        treemodel <- rpart(Pct_Chg ~ ., method="anova",data = train)
        prunedtree <- prune(treemodel,
                    treemodel$cptable[which.min(treemodel$cptable[,'xerror']),"CP"])
        AICpruned.test <- predict(prunedtree,test[,-1])
        AICpruned.TE <- mean((test[,1]-AICpruned.test)^2)
        AICpruned.TE_list <- c(AICpruned.TE_list,AICpruned.TE)  
    }

    # AIC Random Forest
    set.seed(19890211)
    AICforest.TE_list <- vector(length=0)
    for(i in 1:B) {
        start <- sample(1:(n-n1-1),1)
        flag <- (start+6):(start+n1-6)
        train <- aic_vars[-flag,]
        test <- aic_vars[flag,]

        forestmodel <- randomForest(Pct_Chg ~ ., data = train)
        AICforest.test <- predict(forestmodel,test[,-1])
        AICforest.TE <- mean((test[,1]-AICforest.test)^2)
        AICforest.TE_list <- c(AICforest.TE_list, AICforest.TE) 

    }

    output1 <- c('AIC','AIC Tree','AIC Pruned', 'AIC Forest')
    output1 <- rbind(output1,c('Mean',mean(AICtree.TE_list),mean(AICpruned.TE_list),mean(AICforest.TE_list)))
    output1 <- rbind(output1,c('Variance',var(AICtree.TE_list),var(AICpruned.TE_list),var(AICforest.TE_list)))


    return(output1)
}


print('canada')
test_aic_trees('canada',10)
print('europe')
test_aic_trees('europe',10)
print('mexico')
test_aic_trees('mexico',10)
print('japan')
test_aic_trees('japan',10)
print('korea')
test_aic_trees('korea',10)

[1] "canada"


ERROR: Error in `[.data.frame`(full, var_list): undefined columns selected


[1] "europe"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0103816242516105,0.0104506346548643,0.0124021010473583
,Variance,1.99709815212007e-05,1.91521608093144e-05,5.31634209224944e-05


[1] "mexico"


ERROR: Error in `[.data.frame`(full, var_list): undefined columns selected


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0241485786400307,0.0242219651472977,0.0190773710410276
,Variance,5.39216891991406e-05,5.15751108636021e-05,7.23097547629656e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00942921702920031,0.00949326438061241,0.00720811238193397
,Variance,0.000125754514990017,0.000125489555476,3.11268213505039e-05


In [5]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))


full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00641176429333256,0.0065465547044811,0.00592102732259112
,Variance,5.5251285524091e-05,5.63924147604228e-05,5.15281046833127e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.021517387729752,0.0215261055995086,0.0129708191130335
,Variance,0.000195337155232761,0.000181170412091388,6.2762800379708e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00566359308998456,0.0055931082863976,0.00345560664662473
,Variance,1.72954007532264e-05,1.67116080415411e-05,9.31812349670489e-06


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.033376280576257,0.0332824737418788,0.0236266491650808
,Variance,0.000252324471835863,0.000252883012459207,9.71862810487227e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0145229900137072,0.0145205879900288,0.00856645783845349
,Variance,9.50705703903392e-05,9.45136995809354e-05,9.42903210880554e-05


In [6]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00667886751046769,0.00667092755628869,0.00593793699290156
,Variance,5.37516344168912e-05,5.34013552251048e-05,5.00421390854491e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0206253139444022,0.0205606951851182,0.0136384271967105
,Variance,0.000167544939892893,0.000168455265277135,6.14278105108876e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00560876917290206,0.00550172304608404,0.00460318416430582
,Variance,2.37480506055359e-05,2.18911356610622e-05,2.92508824097789e-05


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0375041033727014,0.0375621867049354,0.0241980286282959
,Variance,0.000410730763302962,0.000424241350685863,0.000110603114136601


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.014100728593341,0.0140096080130799,0.00718709750764093
,Variance,9.96637363364853e-05,9.99473524602808e-05,6.76180906516778e-05


In [8]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00633638793798527,0.00621977331115025,0.0048794289887415
,Variance,4.82808774944263e-05,4.58837573632282e-05,3.45100540877489e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.019414823008102,0.0198637424860631,0.0149496372429318
,Variance,0.000124531053160331,0.000146740880463341,7.91779814850825e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00675584774646629,0.00648876151961417,0.00566365139596594
,Variance,2.23212403385359e-05,2.15503427442339e-05,3.2637064296227e-05


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0352526249919635,0.0353772815439313,0.0241965965505857
,Variance,0.000330508942764121,0.000333081431385631,0.000123719964564766


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0158511860221116,0.0158024127430073,0.00879375733892992
,Variance,0.000102645035548097,0.000104306862244824,7.89547067188721e-05


In [9]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00680462829068305,0.00687922575313016,0.00457214475174723
,Variance,6.60472535202397e-05,6.65188037837501e-05,3.24506608638021e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0227619834671656,0.0227924600210191,0.0150212541682689
,Variance,0.000186138166449033,0.000193379926449185,8.59728851011119e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0070837145777984,0.00691456032435144,0.0057785952347072
,Variance,2.70034712708398e-05,2.7028240692858e-05,4.36514469579504e-05


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0330721868624739,0.0330195971992459,0.0218985084393031
,Variance,0.000288205113671281,0.000291853003529252,7.9319272208785e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0119839174585882,0.0119757402523155,0.00879097966967551
,Variance,9.64884973850864e-05,9.75991560001485e-05,9.46950699156657e-05


In [10]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00738881355527953,0.00734461551975625,0.00449553836717566
,Variance,6.81083472834278e-05,6.63864784116058e-05,3.53430112402808e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0217281710690849,0.0219035232598653,0.0143217478868706
,Variance,0.000180664463967327,0.000183393530030901,9.1887159221236e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00639137793224928,0.00602320275556258,0.00404840897692636
,Variance,1.65223040486217e-05,1.62217860976515e-05,1.07588451486262e-05


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.026292644842399,0.0260870577934918,0.0202909203876781
,Variance,0.000104323242956075,0.000102268321690948,6.90613100201775e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.012729679550198,0.0128071321914963,0.00966755904843823
,Variance,7.43913582202799e-05,7.38776897245598e-05,0.000111925098700196


In [11]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00632818371050797,0.00633556804805361,0.00572198516599678
,Variance,5.11763283085318e-05,5.07342179348371e-05,6.15439822029388e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0234175207719483,0.0233819695564869,0.0157595637793029
,Variance,0.000270758611027842,0.000266204535785848,0.000111320044988505


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00620051397700545,0.00621235669664418,0.00313367347085067
,Variance,2.82760239485212e-05,2.95301316308539e-05,6.37894489359502e-06


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0271566346099998,0.0269836296865464,0.0206899863938671
,Variance,0.000107655433766088,0.000102997027021231,6.59539813342089e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0142880214202198,0.0143698389666439,0.009413044993738
,Variance,8.82222133241143e-05,8.62851260314784e-05,0.000108119653996643


In [12]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0071538424936152,0.00717573546335076,0.0052191942049831
,Variance,8.53476302772296e-05,8.62813892814601e-05,5.61641551625972e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0197017431857603,0.0200090359426321,0.0138535853907209
,Variance,7.48057072340353e-05,8.17324355308687e-05,8.04226905762999e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00646410551679496,0.00636401477026538,0.00408503571876879
,Variance,2.61671551688303e-05,2.7565087369239e-05,1.66200541745753e-05


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0286944341041191,0.0284888042286558,0.0213237366978958
,Variance,0.000298914275070719,0.000298721717368116,6.17467837586999e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0127518374340552,0.012817220658813,0.00933978269579364
,Variance,9.07184186363705e-05,9.02170593619813e-05,0.000115939827889348


In [13]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6)) %>%

                mutate(lag7.Exchange = lag(Exchange,7)) %>%
                mutate(lag7.X1Y_Yield_d = lag(X1Y_Yield_d,7)) %>%
                mutate(lag7.X1Y_Yield_f = lag(X1Y_Yield_f,7)) %>%
                mutate(lag7.Infl_d = lag(Infl_d,7)) %>%
                mutate(lag7.Infl_f = lag(Infl_f,7)) %>%
                mutate(lag7.Int_d = lag(Int_d,7)) %>%
                mutate(lag7.Int_f = lag(Int_f,7))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00554410732563508,0.00548976639830343,0.00522782391276848
,Variance,3.86600009386317e-05,3.92117883312043e-05,6.26140253121961e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0180673327445436,0.0183763931652427,0.0131285449600935
,Variance,8.58200123166978e-05,9.0641715187494e-05,8.07190210296585e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00655824984924581,0.00660429565925371,0.00404790159244874
,Variance,5.06912283698866e-05,5.0841478397267e-05,1.39003728685671e-05


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0273959655431472,0.0274043770667193,0.0207074672264812
,Variance,0.000189102781631345,0.000188695248355497,6.8523212565192e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0145341481344421,0.0148013983457763,0.0096775904213935
,Variance,8.39074058922083e-05,8.75664823320566e-05,0.000113683817060146


In [14]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6)) %>%

                mutate(lag7.Exchange = lag(Exchange,7)) %>%
                mutate(lag7.X1Y_Yield_d = lag(X1Y_Yield_d,7)) %>%
                mutate(lag7.X1Y_Yield_f = lag(X1Y_Yield_f,7)) %>%
                mutate(lag7.Infl_d = lag(Infl_d,7)) %>%
                mutate(lag7.Infl_f = lag(Infl_f,7)) %>%
                mutate(lag7.Int_d = lag(Int_d,7)) %>%
                mutate(lag7.Int_f = lag(Int_f,7)) %>%

                mutate(lag8.Exchange = lag(Exchange,8)) %>%
                mutate(lag8.X1Y_Yield_d = lag(X1Y_Yield_d,8)) %>%
                mutate(lag8.X1Y_Yield_f = lag(X1Y_Yield_f,8)) %>%
                mutate(lag8.Infl_d = lag(Infl_d,8)) %>%
                mutate(lag8.Infl_f = lag(Infl_f,8)) %>%
                mutate(lag8.Int_d = lag(Int_d,8)) %>%
                mutate(lag8.Int_f = lag(Int_f,8))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00681947885573587,0.00691188993265094,0.00455615419014244
,Variance,8.25743468710592e-05,8.13808267352957e-05,3.94297313355836e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0191725249021173,0.0192645404191145,0.0134108666692565
,Variance,0.000127978426107884,0.000126700807904745,6.1815314894508e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00421825470391673,0.00424706058672509,0.00441892712788619
,Variance,3.11533130345124e-05,3.16643413437764e-05,1.67473017317252e-05


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0288610001528075,0.0288556658898031,0.0194989468913627
,Variance,0.00018559439006926,0.000186538193993641,4.73702379712535e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0148803495025235,0.0144179469880041,0.00902913828954807
,Variance,9.80123783913825e-05,0.000100931057112197,0.000102209277399995


In [15]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6)) %>%

                mutate(lag7.Exchange = lag(Exchange,7)) %>%
                mutate(lag7.X1Y_Yield_d = lag(X1Y_Yield_d,7)) %>%
                mutate(lag7.X1Y_Yield_f = lag(X1Y_Yield_f,7)) %>%
                mutate(lag7.Infl_d = lag(Infl_d,7)) %>%
                mutate(lag7.Infl_f = lag(Infl_f,7)) %>%
                mutate(lag7.Int_d = lag(Int_d,7)) %>%
                mutate(lag7.Int_f = lag(Int_f,7)) %>%

                mutate(lag8.Exchange = lag(Exchange,8)) %>%
                mutate(lag8.X1Y_Yield_d = lag(X1Y_Yield_d,8)) %>%
                mutate(lag8.X1Y_Yield_f = lag(X1Y_Yield_f,8)) %>%
                mutate(lag8.Infl_d = lag(Infl_d,8)) %>%
                mutate(lag8.Infl_f = lag(Infl_f,8)) %>%
                mutate(lag8.Int_d = lag(Int_d,8)) %>%
                mutate(lag8.Int_f = lag(Int_f,8)) %>%
    
                mutate(lag9.Exchange = lag(Exchange,9)) %>%
                mutate(lag9.X1Y_Yield_d = lag(X1Y_Yield_d,9)) %>%
                mutate(lag9.X1Y_Yield_f = lag(X1Y_Yield_f,9)) %>%
                mutate(lag9.Infl_d = lag(Infl_d,9)) %>%
                mutate(lag9.Infl_f = lag(Infl_f,9)) %>%
                mutate(lag9.Int_d = lag(Int_d,9)) %>%
                mutate(lag9.Int_f = lag(Int_f,9))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00688464601939339,0.00692051215770299,0.00499734076862369
,Variance,6.78283238872792e-05,6.73627278820789e-05,3.91746678462974e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.018559912364554,0.0186502027373072,0.0117277801853307
,Variance,0.000155115258102305,0.000156048693641692,5.10149439376311e-05


[1] "mexico"


...

output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00588881746667127,0.00588510911453325,0.00385358320812958
,Variance,3.14794576669526e-05,2.82456954536241e-05,1.66657651173275e-05


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0321145869754097,0.0319260536104283,0.0184043480877135
,Variance,0.000253638592693912,0.0002472665555028,4.32466244040458e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00825814125651387,0.0084239273887678,0.00947632468592924
,Variance,6.80557341410091e-05,6.60906403996776e-05,8.65451375568666e-05


In [16]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6)) %>%

                mutate(lag7.Exchange = lag(Exchange,7)) %>%
                mutate(lag7.X1Y_Yield_d = lag(X1Y_Yield_d,7)) %>%
                mutate(lag7.X1Y_Yield_f = lag(X1Y_Yield_f,7)) %>%
                mutate(lag7.Infl_d = lag(Infl_d,7)) %>%
                mutate(lag7.Infl_f = lag(Infl_f,7)) %>%
                mutate(lag7.Int_d = lag(Int_d,7)) %>%
                mutate(lag7.Int_f = lag(Int_f,7)) %>%

                mutate(lag8.Exchange = lag(Exchange,8)) %>%
                mutate(lag8.X1Y_Yield_d = lag(X1Y_Yield_d,8)) %>%
                mutate(lag8.X1Y_Yield_f = lag(X1Y_Yield_f,8)) %>%
                mutate(lag8.Infl_d = lag(Infl_d,8)) %>%
                mutate(lag8.Infl_f = lag(Infl_f,8)) %>%
                mutate(lag8.Int_d = lag(Int_d,8)) %>%
                mutate(lag8.Int_f = lag(Int_f,8)) %>%
    
                mutate(lag9.Exchange = lag(Exchange,9)) %>%
                mutate(lag9.X1Y_Yield_d = lag(X1Y_Yield_d,9)) %>%
                mutate(lag9.X1Y_Yield_f = lag(X1Y_Yield_f,9)) %>%
                mutate(lag9.Infl_d = lag(Infl_d,9)) %>%
                mutate(lag9.Infl_f = lag(Infl_f,9)) %>%
                mutate(lag9.Int_d = lag(Int_d,9)) %>%
                mutate(lag9.Int_f = lag(Int_f,9)) %>%
    
                mutate(lag10.Exchange = lag(Exchange,10)) %>%
                mutate(lag10.X1Y_Yield_d = lag(X1Y_Yield_d,10)) %>%
                mutate(lag10.X1Y_Yield_f = lag(X1Y_Yield_f,10)) %>%
                mutate(lag10.Infl_d = lag(Infl_d,10)) %>%
                mutate(lag10.Infl_f = lag(Infl_f,10)) %>%
                mutate(lag10.Int_d = lag(Int_d,10)) %>%
                mutate(lag10.Int_f = lag(Int_f,10)) 

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}

print('canada')
test_full_trees('canada',100)
print('europe')
test_full_trees('europe',100)
print('mexico')
test_full_trees('mexico',100)
print('japan')
test_full_trees('japan',100)
print('korea')
test_full_trees('korea',100)

print('canada')
test_aic_trees('canada',100)
print('europe')
test_aic_trees('europe',100)
print('mexico')
test_aic_trees('mexico',100)
print('japan')
test_aic_trees('japan',100)
print('korea')
test_aic_trees('korea',100)

[1] "canada"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.00563409394203227,0.00564438212499927,0.00461066777799263
,Variance,4.54186956112902e-05,4.50671307406396e-05,3.96442143293759e-05


[1] "europe"


output1,Full,Full Tree,Full Pruned,Full Forest
,Mean,0.0151217018496651,0.0153012434127315,0.0113965787755688
,Variance,6.97866391727101e-05,7.22146592991335e-05,4.93611502963855e-05


[1] "mexico"


...

ERROR: Error in `[.data.frame`(full, var_list): undefined columns selected


[1] "japan"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.0296440302395559,0.0296440172972504,0.0190270215807868
,Variance,0.000174909431687258,0.000174266383257628,4.9186765041553e-05


[1] "korea"


output1,AIC,AIC Tree,AIC Pruned,AIC Forest
,Mean,0.00958579691676819,0.00960645211932238,0.00775683615187804
,Variance,9.81619074671622e-05,9.77585523131729e-05,5.64308796330171e-05
